In [ ]:
from pprint import pprint
import json

with open('bias-study-export.json') as f:
    data = json.load(f)['logs']
    
with open('images/_airlines.json') as f:
    airlines = json.load(f)

with open('images/_states.json') as f:
    states = json.load(f)

pprint(data['82534293'])

In [ ]:
from collections import OrderedDict 
from copy import deepcopy

rows = []

for p in data.items():
    d = OrderedDict({
        'id': p[0]
    })
    
    item = p[1]
    
    d['userAgent'] = item['userAgent']
    d['isMobile'] = item['isMobile']
    d['done'] = 'done' in item
    
    cond = item['cond']
    
    errorAirline = cond['errorAirline']
    seqAirline = cond['seqAirline']
    errorStates = cond['errorStates']
    seqStates = cond['seqStates']
    focusAirline = cond['focusAirline']
    focusState = cond['focusState']
    
    for k, v in cond.items():
        d[k] = v
        
    howManyIndex = 0
    for k, v in item['approximateVis_airline'].items():
        t = v['type']
        
        truth = airlines["{}_{}".format(errorAirline, seqAirline)]
        
        prefix = 'approx.airline.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'true'] = truth[focusAirline]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'true'] = truth[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
    
    howManyIndex = 0
    for k, v in item['approximateVis_states'].items():
        t = v['type']
        
        truth = states["{}_{}".format(errorStates, seqStates)]
        
        prefix = 'approx.states.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'true'] = truth[focusState]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'true'] = truth[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
     
    def jaccard_similarity(x,y):
        intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
        union_cardinality = len(set.union(*[set(x), set(y)]))
        return intersection_cardinality/float(union_cardinality)
    
    # precise
            
    howManyIndex = 0
    for k, v in item['preciseVis_airline'].items():
        t = v['type']
        
        precise = airlines["{}_{}".format(1, 1)]
        approx = airlines["{}_{}".format(errorAirline, seqAirline)]
        
        prefix = 'precise.airline.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'true'] = precise[focusAirline]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'true'] = precise[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
            
        if (t == 'DidYouNotice'):
            p = prefix + 'didYouNotice.'
            d[p + 'answer'] = v['answer'] == 'yes'
            d[p + 'true'] = errorAirline > 1
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'SelectAll'):
            v = deepcopy(v)
            p = prefix + 'SelectAll.'
            d[p + 'confidence'] = v['confidence']
            
            del v['confidence']
            del v['type']
            
            answers = v.keys()
            prec = [k for k,v in precise.items() if v]
            appr = [k for k,v in approx.items() if v]
            d[p + 'answer'] = ','.join(answers)
            d[p + 'approx'] = ','.join(appr)
            d[p + 'precise'] = ','.join(prec)
            d[p + 'jaccard_answer_precise'] = jaccard_similarity(answers, prec)
            
    howManyIndex = 0
    for k, v in item['preciseVis_states'].items():
        t = v['type']
        
        precise = states["{}_{}".format(1, 1)]
        approx = states["{}_{}".format(errorStates, seqStates)]
        
        prefix = 'precise.states.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'true'] = precise[focusState]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'true'] = precise[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
            
        if (t == 'DidYouNotice'):
            p = prefix + 'didYouNotice.'
            d[p + 'answer'] = v['answer'] == 'yes'
            d[p + 'true'] = errorStates > 1
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'SelectAll'):
            v = deepcopy(v)
            p = prefix + 'SelectAll.'
            d[p + 'confidence'] = v['confidence']
            
            del v['confidence']
            del v['type']
            
            answers = v.keys()
            prec = [k for k,v in precise.items() if v]
            appr = [k for k,v in approx.items() if v]
            d[p + 'answer'] = ','.join(answers)
            d[p + 'approx'] = ','.join(appr)
            d[p + 'precise'] = ','.join(prec)
            d[p + 'jaccard_answer_precise'] = jaccard_similarity(answers, prec)
    
    for k, v in item['timing'].items():
         d['timing.' + k] = v
            
    for k, v in item['demographics'].items():
         d['demographics.' + k] = v
    
    rows.append(d)

pprint(rows)

In [ ]:
import csv

with open('bias-study-export.csv', 'w') as f:
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    
    for row in rows:
        writer.writerow(row)

        
import pandas as pd

pd.read_csv("bias-study-export.csv")